In [37]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

Sample_File = tf.io.read_file('/Users/nicolahaller/Documents/WI Master/Semester 2/Technology Lab/Samples/Tea K Pea - nauticals_1.wav')
Sample_File, _ = tf.audio.decode_wav(contents=Sample_File)
print(Sample_File.shape)
print(Sample_File)

'''model = models.Sequential()
model.add(layers.Conv2D(32, (44100, 2), strides=(1,1), padding="valid", activation='relu', input_shape=(44100, 2))
model.add(layers.MaxPooling2D((22050, 2)))
model.add(layers.Conv2D(64, (44100, 2), activation='relu'))
model.add(layers.MaxPooling2D((22050, 2)))
model.add(layers.Conv2D(64, (44100, 2), activation='relu'))
'''
model = models.Sequential()
model.add(layers.Conv1D(32, (44100), strides=(1,1), padding="valid", activation='relu', input_shape=(44100, 2)))
model.add(layers.MaxPool1D(22050))
model.add(layers.Conv1D(64, (44100), activation='relu'))
model.add(layers.MaxPool1D(22050))
model.add(layers.Conv1D(64, (44100), activation='relu'))

print(model.summary())

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(Sample_File, epochs=10)

(44100, 2)
tf.Tensor(
[[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 ...
 [0.02478027 0.01925659]
 [0.0246582  0.01849365]
 [0.02456665 0.0178833 ]], shape=(44100, 2), dtype=float32)


ValueError: The `strides` argument must be a tuple of 1 integers. Received: (1, 1)

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length)])
X = seq.reshape(len(seq), 1, 1)
y = seq.reshape(len(seq), 1)
# define LSTM configuration
n_neurons = length
n_batch = length
n_epoch = 1000
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train LSTM
model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=2)
# evaluate
result = model.predict(X, batch_size=n_batch, verbose=0)
for value in result:
	print('%.1f' % value)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5)                 140       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
1/1 - 2s - loss: 0.3157 - 2s/epoch - 2s/step
Epoch 2/1000
1/1 - 0s - loss: 0.3137 - 6ms/epoch - 6ms/step
Epoch 3/1000
1/1 - 0s - loss: 0.3118 - 3ms/epoch - 3ms/step
Epoch 4/1000
1/1 - 0s - loss: 0.3098 - 5ms/epoch - 5ms/step
Epoch 5/1000
1/1 - 0s - loss: 0.3079 - 13ms/epoch - 13ms/step
Epoch 6/1000
1/1 - 0s - loss: 0.3059 - 5ms/epoch - 5ms/step
Epoch 7/1000
1/1 - 0s - loss: 0.3040 - 3ms/epoch - 3ms/step
Epoch 8/1000
1/1 - 0s - loss:

In [ ]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))
# make a prediction
row = asarray([1, 2, 3, 4, 5]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))